### Run this in Python kernel

In [ ]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            [clojure-backtesting.parameters :refer :all]
            [clojure-backtesting.large-data :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojure.java.io :as io]
            [clojure.data.csv :as csv]
            [clojupyter.kernel.version :as ver]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [2]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative to your own dataset
;
(load-large-dataset "/Users/lyc/Downloads/data-sorted/data-CRSP-sorted.csv" "main")
(set-main "main")

true

In [3]:
(time (while 
    (< (compare (curr-date) "1990-02") 0)(next-day)))

"Elapsed time: 595183.231167 msecs"


nil

In [11]:
(time (while 
    (< (compare (curr-date) "2016-02") 0)(next-day)))

"Elapsed time: 916551.674745 msecs"


nil

In [18]:
(defn orderlazy 
;;  ([quantity & {:keys [remaining leverage print direct] :or {remaining false leverage LEVERAGE print false direct true}}]
;;   (order-lazy (curr-tic) quantity :remaining remaining :leverage leverage :print print :direct direct))
 ([tic quantity & {:keys [expiration remaining leverage dataset print direct] :or {expiration ORDER-EXPIRATION remaining false leverage LEVERAGE dataset (deref data-set) print false direct true}}]
  (swap! pending-order assoc tic {:date (curr-date) :expiration expiration :quantity quantity :remaining remaining :leverage leverage :print print :direct direct})))


#'clojure-backtesting.demo/orderlazy

In [30]:
(defn search-inorder
  "This function turns the order processed date"
  [date tic dataset]
;; date e.g. "DD/MM/YYYY"
;; tic e.g. "AAPL"
;; return [false "No match date" 0 0 0] if no match
;; return [true T+1-date price aprc reference] otherwise
  (if (and (not (deref lazy-mode)) (not= (count (deref available-tics-)) 0))
    (if (and (not= -1 (.indexOf (keys (deref available-tics-)) tic)) (not= (get (get (get (deref available-tics-) tic) :reference) :date) (get (get (deref tics-info) tic) :end-date)))
      (let [t-1-date (get (first (rest (get (get (deref available-tics-) tic) :reference))) :date)
            [b p aprc r] (search-date t-1-date tic (get (get (deref available-tics-) tic) :reference))]
        (if b
          [b t-1-date (Double/parseDouble p) aprc r]))
      [false "No such date or ticker in the dataset or the dataset has reached the end" 0 0 0])
    (let [[match price aprc reference] (search-date date tic dataset)]
		;;(let [[match price reference] [true "10" 348]]
      (if match
        (if (deref lazy-mode) 
            [match price aprc reference]
            (loop [i 1]
          (if (<= i MAXLOOKAHEAD)
            (let [t-1-date (look-ahead-i-days date i)
                  [b p aprc r] (search-date t-1-date tic dataset)]
              (if b
                [b t-1-date (Double/parseDouble p) aprc r]
                (recur (inc i))))
            [false (str "No appropriate order date after looking ahead " MAXLOOKAHEAD " days") 0 0 0])))
        [false "No such date or ticker in the dataset" 0 0 0]))))

#'clojure-backtesting.demo/search-inorder

In [39]:
(search-date "2016-02-01" "A" (get (deref dataset-col) "main"))

[true "37.69" 37.707378190778606 0]

In [40]:
(first [(get-line)])

{:ewretd "-0.001282", :HSICCD "3826", :CUM-RET 4.6097600482540737E-4, :CFACSHR "1.0", :date "2016-02-01", :INIT-PRICE 37.69, :OPENPRC "37.37", :SECSTAT "R", :SHROUT "327.769", :TICKER "A", :APRC 37.707378190778606, :COMNAM "AGILENT TECHNOLOGIES INC", :PRIMEXCH "N", :TRDSTAT "A", :FACSHR "", :HEXCD "1", :LOG-RET 4.6097600482540737E-4, :RET "0.001062", :EXCHCD "1", :CFACPR "1.0", :DLRET "", :PRC "37.69", :vwretd "-0.00035800000000000003", :FACPR "", :CUSIP "00846U10", :NCUSIP "00846U10", :PERMCO "36364", :DIVAMT "", :PERMNO "87432", :SHRCD "11", :sprtrn "-0.000443", :VOL "2666567.0", :SICCD "3825"}

### Initialise portfolio （Go back here everytime you want to restart.）

In [ ]:
;; initialise with current date and initial capital (= $10000)
(init-portfolio "1980-12-15" 100000);

In [ ]:
(count (deref tics-info))

### Write a strategy

The following code implements a trading strategy called Golden Rule:

MA 50 cross above the MA 200 (golden cross)

MA 200 cross below the MA 50 (death cross)

So in the codes, MA50 and MA200 are compared on a daily basis, if golden cross occurs, then you set a buy order; if death cross occurs, then you set a sell order first 



In [ ]:
(time (do (def MA50-vec-aapl [])
          (def MA200-vec-aapl [])
          (def MA50-vec-f [])
          (def MA200-vec-f [])
          (while (not= (get-date) "2016-12-29")
            (do
    ;; write your trading strategy here
              (def tics (deref available-tics-)) ;20 ms
              (def MA50-vec-aapl (get-prev-n-days :PRC 50 "AAPL" MA50-vec-aapl (get (get tics "AAPL"):reference)))
              (def MA200-vec-aapl (get-prev-n-days :PRC 200 "AAPL" MA200-vec-aapl (get (get tics "AAPL") :reference)))
              (def MA50-vec-f (get-prev-n-days :PRC 50 "F" MA50-vec-f (get (get tics "F"):reference)))
              (def MA200-vec-f (get-prev-n-days :PRC 200 "F" MA200-vec-f (get (get tics "F") :reference)))
              (let [[MA50 MA200] [(moving-average :PRC MA50-vec-aapl) (moving-average :PRC MA200-vec-aapl)]]
                (if (> MA50 MA200)
                  (order "AAPL" 1 :reference (get (get tics "AAPL") :reference) :print false) 
                  (order "AAPL" 0 :remaining true :reference (get (get tics "AAPL") :reference))))
              (let [[MA50 MA200] [(moving-average :PRC MA50-vec-f) (moving-average :PRC MA200-vec-f)]]
                (if (> MA50 MA200)
                  (order "F" 1 :reference (get (get tics "F") :reference) :print false) 
                  (order "F" 0 :remaining true :reference (get (get tics "F") :reference))))
              ;(update-eval-report (get-date))
              (next-date)))))
(.close wrtr)

In [ ]:
(count (deref order-record))


### Check portfolio record

In [ ]:
;; view final portfolio
(view-portfolio)

In [ ]:
;; view portfolio value and return
(view-portfolio-record)

### Generate evaluation report

In [ ]:
(eval-report)

### Plot variables

In [ ]:
(def data (deref portfolio-value))

In [ ]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "portfolio")
  data))

In [ ]:
(first data-to-plot)

In [ ]:
(plot data-to-plot :plot :date :daily-ret)